In [7]:
#Flow
# problems: 1. video lag 2. stream end 3. anti-shake 4. simulation of the method 
# refresh parameters 1. change in 'parameters.ini' 2. press 'r'
# to do log information, max position and angle, average angle
import numpy as np 
import cv2 as cv
from numpy.core.numeric import zeros_like
import colorsys
from vidgear.gears import VideoGear
import datetime
import os
import csv


print("import success!")

import success!


In [8]:
# 注意winsize、roi_region、结构的大小比例、结构的密度、结构和winsize的比例

# hsv transform to rgb format
def hsv2rgb(h,s,v):
    return tuple(round(i * 255) for i in colorsys.hsv_to_rgb(h,s,v))

#control parameters
is_save_stream = False
is_ref_refresh = False
is_anti_shake = False

# default parameters  
# calculation parameters
winsize = 64
mag_ceiling = 0.5
mag_floor = mag_ceiling/5

# camer & image parameters
alpha = 0.8 
hue = 90                     
RES = (2592, 1944)                 # camera resolution
# RES = (1920, 1080)
# roi_rect = [0, RES[0], 0, RES[1]]
roi_rect = [0+100,RES[0]-100,0+100,RES[1]-100]  # region of interesting
# roi_rect = [int(RES[0]*0.4), int(RES[0]*0.6), int(RES[1]*0.4), int(RES[1]*0.6)]

result_RES = (640, 480)         # flow image resolution

# stable calculation parameters
pyr_scale = 0.5
levels = 2
iterations = 1
poly_n = 5
poly_sigma = 1.1
flags = cv.OPTFLOW_FARNEBACK_GAUSSIAN

# drawing buffer
ax = []
ay1 = []
ay2 = []

# formatting parameters as dictionary attributes
options = {"CAP_PROP_FRAME_WIDTH":2592, "CAP_PROP_FRAME_HEIGHT":1944, "CAP_PROP_FPS":30}

print("initiation success")

initiation success


In [9]:
# 水平噪声、数值噪声
# stream = VideoGear(source=1, resolution=RES, stabilize = True).start()
stream = VideoGear(source=1, resolution=RES, stabilize = True, **options).start()

print("stream start!")

# pre-read frame
frame_pre = stream.read()
bgr_pre = cv.cvtColor(frame_pre, cv.COLOR_BGR2GRAY)

# initial hsv
hsv = np.zeros_like(frame_pre[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]])  
hsv[...,1] = 255 #saturation is full
blank = np.zeros_like(frame_pre)

# video loop
count = 0
print("begin stream!")


stream start!
begin stream!


In [10]:
# 测试区
print("hsv shape is ", hsv.shape)
print("bgr_gre shape is ", bgr_pre.shape)
print("video resolution is (height, width, channel) : ",frame_pre.shape)
print("roi_rect is ", roi_rect)

# 光流算法、PIV算法原理

hsv shape is  (1744, 2392, 3)
bgr_gre shape is  (1944, 2592)
video resolution is (height, width, channel) :  (1944, 2592, 3)
roi_rect is  [100, 2492, 100, 1844]


In [11]:
# 程序执行区域
timeNow = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
os.makedirs("tVariance_" + timeNow)
with open("tVariance_" + timeNow + '/Variance.csv','w',newline='')as csv_file:
  while(True):
  # read current frame from cap
      frame_cur = stream.read()
      bgr_cur = cv.cvtColor(frame_cur, cv.COLOR_BGR2GRAY) # change in to gray

      roi_bgr_pre = bgr_pre[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]]
      roi_bgr_cur = bgr_cur[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]]

      # calculate flow
      flow = cv.calcOpticalFlowFarneback(roi_bgr_pre,roi_bgr_cur, None,
                                      pyr_scale,
                                      levels,
                                      winsize,
                                      iterations,
                                      poly_n,
                                      poly_sigma,
                                      flags)
      mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])  # orginal flow
    
      mag_mean = cv.mean(mag)[0]
      mag_sft = abs(mag - mag_mean)  # shifted magnitude to elimiate noise
      hsv[...,0] = (ang + hue)*180/np.pi/2 # color space related to angle 
      hsv[...,2] = cv.normalize(mag_sft,None,0,255,cv.NORM_MINMAX) 
      
      #enhanced flow
      bgr_flow = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
      mag_enhanced = zeros_like(mag_sft)
      cv.min(mag_sft,mag_ceiling,mag_enhanced)  # enhance flow, ceiling and flooring
      cv.max(mag_enhanced,mag_floor,mag_enhanced)
      hsv[...,0] = (ang + hue)*180/np.pi/2 # color space related to angle 
      hsv[...,2] = cv.normalize(mag_enhanced,None,0,255,cv.NORM_MINMAX)
      bgr_flow_enhanced = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
      
      # image emerge with enhanced flow
      flow_blend_enhance = cv.addWeighted(frame_cur[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]], 1-alpha ,bgr_flow_enhanced,  alpha, 0)
      bgr_blend = cv.addWeighted(frame_cur[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]], 1-alpha ,bgr_flow,  alpha, 0)
      frame_blend = cv.addWeighted(frame_cur, 1-alpha ,blank,  alpha, 0)#frame_cur.copy()
      frame_blend[roi_rect[2]:roi_rect[3],roi_rect[0]:roi_rect[1]] = flow_blend_enhance

      # if ture, compare image is  successive; if false, compare image is not change
      if is_ref_refresh:
        bgr_pre = bgr_cur
    
      #palette
      palette=np.zeros((512,512,3),np.uint8)
      mag_norm = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
      mag_min,mag_max,min_indx,max_indx=cv.minMaxLoc(mag)
      ang_mean = cv.mean(ang*mag/mag_mean)[0]

      # 计算水平噪声和竖直噪声
      hor_Noise = np.average(np.multiply(mag, np.sin(ang)))
      ver_Noise = np.average(np.multiply(mag, np.cos(ang)))
      # 绝对值计算水平噪声和竖直噪声
      hor_AbsNoise = np.average(np.abs(np.multiply(mag, np.sin(ang))))
      ver_AbsNoise = np.average(np.abs(np.multiply(mag, np.cos(ang))))


      cv.putText(palette, "max="+str(mag.max()), (0, 30), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
      cv.putText(palette, "mag_mean="+str("%.2f"%mag_mean), (0, 60), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
      cv.putText(palette, "mag_max="+str(mag_max), (0, 90), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
      cv.putText(palette, "mag_min="+str(mag_min), (0, 120), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
      cv.putText(palette, "hor_Noise="+str(hor_Noise), (0, 150), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
      cv.putText(palette, "ver_Noise="+str(ver_Noise), (0, 180), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
      cv.putText(palette, "hor_AbsNoise="+str(hor_AbsNoise), (0, 210), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
      cv.putText(palette, "ver_AbsNoise="+str(ver_AbsNoise), (0, 240), cv.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)

      timeNow1 = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
      # max min min variance 
      # 获取一个csv对象进行内容写入
      writer=csv.writer(csv_file)
      # writerow 写入一行数据
      writeDatum = [timeNow1, mag_mean, mag_max, mag_min, hor_Noise, ver_Noise, hor_AbsNoise, ver_AbsNoise]
      writer.writerow(writeDatum)

      count = count + 1
  
      # result flow image
      cv.imshow('Orignal',cv.resize(frame_cur, (result_RES[0], result_RES[1])))
      cv.imshow("Palette", palette)
      cv.imshow('Blender', cv.resize(bgr_blend , (result_RES[0], result_RES[1])))
      cv.imshow('Area of Intrest Blended', cv.resize(frame_blend  , (result_RES[0], result_RES[1])))
      
      key = cv.waitKey(10)
      if key==114: # if input key 'r', refresh compare image
          #打印时间戳保存
          timeNow1 = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
          os.makedirs("Refresh_" + timeNow1)
          bgr_pre = bgr_cur
          # cv.imwrite(timeNow +'/Orignal.jpg',frame_cur)
          # cv.imwrite(timeNow +'/Area_of_Intrest_Blended.jpg', frame_blend)
          cv.imwrite(timeNow1 +"/Palette.jpg", palette)
          cv.imwrite(timeNow1 +'/Blender.jpg', bgr_blend)
          print("the image is refreshed")


        


KeyboardInterrupt: 